Following [Pomegranate tutorial](https://pomegranate.readthedocs.io/en/latest/tutorials/B_Model_Tutorial_4_Hidden_Markov_Models.html)

In [2]:
import numpy as np
import torch
import pomegranate, pomegranate.hmm

/home/tom/miniconda3/envs/SAGAconf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Set up the example

In [3]:
N_TRACKS = 3
# e.g. 0 = dead/quiescent, 1 = promoter, 2 = other actives
states = [0, 1, 2]

Generate some sample observation data assuming each state's emission ~ Normal.

Although in principle, a separate emission distribution for every state-track pair, just use a multivariate Normal, of <# of tracks> dimensions, per state.

In [4]:
MEANS = (0.3, 10.0, 6.0)
STDEVS = (0.05, 1.0, 0.5)
true_emitters = []
for mean, stdev in zip(MEANS, STDEVS):
    multivar_mean = torch.full([N_TRACKS], mean)
    multivar_stdev = torch.full([N_TRACKS], stdev)
    true_emitters.append(torch.distributions.Normal(multivar_mean, multivar_stdev))
true_emitters

[Normal(loc: torch.Size([3]), scale: torch.Size([3])),
 Normal(loc: torch.Size([3]), scale: torch.Size([3])),
 Normal(loc: torch.Size([3]), scale: torch.Size([3]))]

In [5]:
for i, emitter in enumerate(true_emitters):
    print(f"Emitter {i}:\n\t{emitter.loc}\n\t{emitter.scale}")

Emitter 0:
	tensor([0.3000, 0.3000, 0.3000])
	tensor([0.0500, 0.0500, 0.0500])
Emitter 1:
	tensor([10., 10., 10.])
	tensor([1., 1., 1.])
Emitter 2:
	tensor([6., 6., 6.])
	tensor([0.5000, 0.5000, 0.5000])


In [6]:
TRUE_START_PROBS = [0.8, 0.1, 0.1]
sum(TRUE_START_PROBS)

1.0

In [7]:
TRUE_TRANS_PROBS = torch.tensor([
    [0.8, 0.1, 0.1],
    [0.1, 0.6, 0.3],
    [0.3, 0.1, 0.6]
])

In [8]:
SEQ_LEN = 20
gen_seq = np.random.choice(states, size=SEQ_LEN, p=TRUE_START_PROBS)
gen_seq

array([0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 1, 0])

Transpose into column vector.

In [24]:
true_emitters[2].sample().reshape((-1, 1))

tensor([[6.6323],
        [6.0483],
        [6.0506]])

In [35]:
observations = torch.stack([true_emitters[st_id].sample() for st_id in gen_seq])
observations

tensor([[ 0.3054,  0.2229,  0.3701],
        [ 5.5651,  6.5572,  6.5768],
        [ 0.3020,  0.3134,  0.3664],
        [ 0.2947,  0.2546,  0.3192],
        [ 0.2827,  0.2506,  0.2943],
        [10.8162, 10.0738,  8.3960],
        [ 0.2998,  0.4067,  0.3256],
        [ 0.2796,  0.3195,  0.1884],
        [ 0.3502,  0.3037,  0.2364],
        [ 9.7514,  9.8514,  9.9586],
        [ 0.2800,  0.2423,  0.3913],
        [ 9.0129,  9.5638, 10.4782],
        [ 0.3264,  0.3136,  0.3524],
        [ 0.2602,  0.3004,  0.3156],
        [ 0.3349,  0.2948,  0.3594],
        [ 6.4302,  5.2614,  5.5897],
        [ 0.2737,  0.3666,  0.2928],
        [ 0.2977,  0.2984,  0.2823],
        [10.8006, 12.2314,  9.4856],
        [ 0.2861,  0.3369,  0.3937]])

[Tutorial 4 - Hidden Markov Models | Pomegranate](https://pomegranate.readthedocs.io/en/latest/tutorials/B_Model_Tutorial_4_Hidden_Markov_Models.html)
> Similar to other methods, we can create an HMM with uninitialized distributions. These distributions will be __initialized__ using __k-means clustering__ when provided with data. When using a DenseHMM we can also choose to not pass in __edges__ and have them be initialized to __uniform probabilities__.

In [27]:
init_means = torch.full([N_TRACKS], observations.mean().item())
init_vars = torch.diag(torch.ones(N_TRACKS))
model1 = pomegranate.hmm.DenseHMM([pomegranate.distributions.Normal(init_means, init_vars)] * len(states), verbose=True, max_iter=100)
model1

DenseHMM(
  (start): Silent()
  (end): Silent()
)

In [28]:
observations.shape

torch.Size([20, 3])

NOTE: pomegranate's `fit` expects a _sequence_ of observations.

[Fitting a continuous multivariate HMM from labeled data](https://github.com/jmschrei/pomegranate/issues/485#issuecomment-416662827)

In [30]:
model1.fit([observations])

[1] Improvement: 465.9555969238281, Time: 0.004181s
[2] Improvement: 7.62939453125e-06, Time: 0.005155s


DenseHMM(
  (start): Silent()
  (end): Silent()
  (distributions): ModuleList(
    (0): Normal()
    (1): Normal()
    (2): Normal()
  )
)

# Missing Observations
Apparently, pomegranate can handle missing observations as `None`'s in the sequence.

In [38]:
Lobservations = observations.tolist()
Lobservations[0] = [None] * N_TRACKS
Lobservations

[[None, None, None],
 [5.565107345581055, 6.557218551635742, 6.576842784881592],
 [0.3020317852497101, 0.31342706084251404, 0.36642390489578247],
 [0.29472455382347107, 0.2545918822288513, 0.31916943192481995],
 [0.28270891308784485, 0.25064459443092346, 0.2943447232246399],
 [10.81615924835205, 10.073811531066895, 8.395991325378418],
 [0.2997661530971527, 0.4067055284976959, 0.32560163736343384],
 [0.2796158790588379, 0.31951630115509033, 0.18837019801139832],
 [0.3501664400100708, 0.30370476841926575, 0.2363787293434143],
 [9.751447677612305, 9.851372718811035, 9.958648681640625],
 [0.2800060510635376, 0.24227580428123474, 0.39131394028663635],
 [9.012857437133789, 9.563789367675781, 10.47817611694336],
 [0.3264276087284088, 0.31359487771987915, 0.3523578941822052],
 [0.26022782921791077, 0.30037346482276917, 0.3156208395957947],
 [0.334893137216568, 0.29476144909858704, 0.3594101667404175],
 [6.430192470550537, 5.261399269104004, 5.589681148529053],
 [0.2737293243408203, 0.366596549

In [39]:
model1.fit([Lobservations])

RuntimeError: Could not infer dtype of NoneType